In [1]:
import numpy as np
import pandas as pd
# from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV

# Read the data

In [2]:
path = '../../../datasets/objective_subjective/'
subjective_file = 'quote.tok.gt9.5000'

In [3]:
f = open(path + subjective_file, 'r', encoding='utf-8', errors='ignore')
subjective_lines = []
for line in f:
    subjective_lines.append(line)
f.close()

print('Subjective:',len(subjective_lines))
subjective_lines[0]

Subjective: 5000


'smart and alert , thirteen conversations about one thing is a small gem . \n'

In [4]:
objective_file = 'plot.tok.gt9.5000'
f = open(path + objective_file, 'r', encoding='utf-8', errors='ignore')
objective_lines = []
for line in f:
    objective_lines.append(line.rstrip())
f.close()

print('Objective:',len(objective_lines))

Objective: 5000


In [5]:
objective_lines[0]

'the movie begins in the past where a young boy named sam attempts to save celebi from a hunter'

In [6]:
all_lines = objective_lines + subjective_lines
print(len(all_lines))
all_lines[0]

10000


'the movie begins in the past where a young boy named sam attempts to save celebi from a hunter'

## Create tf_idf = f(sentence,term)

In [ ]:
tfidf = TfidfVectorizer() 
sentence_tfidf = tfidf.fit_transform(all_lines)
vocab = tfidf.vocabulary_
if False:
    tfidf_mat = sentence_tfidf
else:
    tfidf_mat = sentence_tfidf.todense()
print(type(tfidf_mat))
print(len(vocab))

In [ ]:
#instead of default TFifV, try:
# where 
word_bag = WordBag()
def dummy_fun(doc):
    return doc

tfidf = TfidfVectorizer(
    analyzer='word',       # Feed a list of words to TF-IDF
    tokenizer=word_bag.comment_to_bag_of_words,
    preprocessor=dummy_fun,
    token_pattern=None,
    lowercase=False, 
    stop_words=None, 
    max_features=MAX_FEATURES,
    norm='l2',            # normalize each review
    use_idf=True)

In [ ]:
pickle.dump(tfidf, open('fit_tfidf_vectorizer_for_obj_subj_sentences_classification.pkl', 'wb'))


In [ ]:
sorted(vocab)[258:268]

In [ ]:
tfidf_mat[:2,270:300]

## Create Train and CV sets

In [ ]:
labels = np.concatenate((np.zeros((5000,), dtype=int), np.ones((5000,), dtype=int)))
print(labels.shape)
labels[4990:5010]

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_cv, y_train, y_cv = train_test_split(
    tfidf_mat, labels, test_size=0.2, random_state=0)

In [ ]:
print(X_train.shape, X_cv.shape, y_train.shape, y_cv.shape)

## Gradient Boosting

In [ ]:
# N_TREES = 300
# LEARN_RATE = 0.2
# MIN_IN_LEAF = 10
# CV score: 0.8285

# 107 trees
# LEARN_RATE = 0.2
# MAX_DEPTH = 8
# MIN_IN_LEAF = 5 #7
# MAX_FEATURES = 'sqrt'
# CV score: 0.8415

# Gradient Boosting Classifier parameters
# N_TREES = int(round(np.sqrt(X_train.shape[0]) * 1.2))
# CV score should be 0.8815
N_TREES = 300
LEARN_RATE = 0.5
MAX_DEPTH = 16
MIN_IN_LEAF = 5 #7
MAX_FEATURES = 'sqrt'
N_TREES

In [ ]:
print(y_train[0])
print(X_train[0])


In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

# loss: deviance: logistic log likelihood
gbc = GradientBoostingClassifier(learning_rate=LEARN_RATE, 
                                   n_estimators=N_TREES, 
                                   min_samples_leaf=MIN_IN_LEAF,
                                   max_features=MAX_FEATURES)
gbc.fit(X_train, y_train)
print('Train score:', gbc.score(X_train, y_train))
print('CV score:', gbc.score(X_cv, y_cv))

In [ ]:
if True:
    grid = {
        'learning_rate': [.01, .05],
        'max_depth': [8, 16],
        'min_samples_leaf': [5],
        'max_features': ['sqrt'],
        'n_estimators': [300],
        'random_state': [0]
    }
else:  # TEST
    grid = {
    'learning_rate': [1],
    'max_depth': [2], 
    'min_samples_leaf': [2],
#     'max_features': ['sqrt', None],
    'n_estimators': [2],
    'random_state': [0]
}
    
# confusion_score = make_scorer(confusion_rmse, greater_is_better=False)

gbc_grid_cv = GridSearchCV(
    GradientBoostingClassifier(), 
    grid,
    cv=4,  # number of folds
    return_train_score=True,
    verbose=1, 
    n_jobs=-1)
gbc_grid_cv.fit(X_train, y_train)

In [ ]:
y_pred = gbc_grid_cv.predict(X_cv)

print(gbc_grid_cv.best_params_)
print(gbc_grid_cv.best_score_)
res_df = pd.DataFrame(gbc_grid_cv.cv_results_)
res_df

In [ ]:
# save the model
import pickle
pickle.dump(gbc, open('GBC_'+ str(N_TREES) +'_' + str(LEARN_RATE) 
                        +'_' + str(MIN_IN_LEAF) + '_20min.pkl', 'wb'))


# LSTM

In [7]:
print(len(all_lines))
all_lines[0]

10000


'the movie begins in the past where a young boy named sam attempts to save celebi from a hunter'

### Find the max length of comments

In [8]:
import sys
sys.path.append('..')

In [22]:
%reload_ext autoreload
from NLP import WordBag
word_bag = WordBag()

In [23]:
cleaned_text = [word_bag.clean_text(sentence, remove_stop_words=False)
                for sentence in all_lines]
                

In [24]:
cleaned_text[3312]

'the course of their experiences ultimately involves with sammy boi   an aggressive rapper caught in a bad contract while on of his songs is a major radio hit   joan vicks   a devious a#   r executive for viewpointe records   randy tigge   a drug addicted   crossdressing rock star   and michael dileo   the chairman of viewpointe record   who some simply call satan '

In [13]:
for i, text in enumerate(cleaned_text):
    if len(text.split()) > 60:
        print(i, text)

81 with his own group   the urban cyber breakers   he has got a dream  to win the dance battle that will bring his team to the world final in new york before reaching usa and fame   he has to overcome the rival group   the ld   to deal with his mothers excessive love and to live out his passion for the beautiful samia 
169 the film captures intense footage of several art events as well as interviews with burning man organizers larry harvey   also cofounder     crimson rose and will roger who explain how   what started in  as an impromptu summer solstice celebration   put on by a few friends at a beach in san francisco   has turned into a fullblown exercise in city planning and management with a hefty pricetag 
243   bums in the mist  will expose you to a world where alcohol is king and sloth is a virtue   buddy george will discover   bums who are able to find love and professional success all while maintaining their  mondern frontiersman  lifestyle   bums whos natural gifts enable them

In [14]:
for i, text in enumerate(all_lines):
    if len(text.split()) > 60:
        print(i, text)

81 with his own group , the urban cyber breakers , he has got a dream : to win the dance battle that will bring his team to the world final in new york before reaching usa and fame , he has to overcome the rival group , the ld , to deal with his mother's excessive love and to live out his passion for the beautiful samia .
169 the film captures intense footage of several art events as well as interviews with burning man organizers larry harvey ( also co-founder ) , crimson rose and will roger who explain how , what started in 1986 as an impromptu summer solstice celebration , put on by a few friends at a beach in san francisco , has turned into a full-blown exercise in city planning and management with a hefty price-tag .
243  " bums in the mist " will expose you to a world where alcohol is king and sloth is a virtue ; buddy george will discover , bums who are able to find love and professional success all while maintaining their " mondern frontiersman " lifestyle ; bums who's natural g

In [ ]:
max([len(text.split()) for text in all_lines])

### Define & run LSTM

In [ ]:
WORD_WINDOW = 50

import tensorflow as tf
keras = tf.keras
lstm = keras.Sequential()
lstm.add(keras.layers.LSTM(32, input_shape=(WORD_WINDOW, 1), return_sequences=True))
lstm.add(keras.layers.LSTM(32, return_sequences=False))
lstm.add(keras.layers.Dense(1, activation='linear'))
lstm.compile(optimizer='rmsprop',
              loss='mse')

In [ ]:
X_train, X_cv, y_train, y_cv = train_test_split(
    all_lines, labels, test_size=0.2, random_state=0)

In [ ]:
model.fit(x_train, y_train, batch_size=32, epochs=10)

## J

In [26]:
MAX_NB_WORDS = 500
max_review_length = 500
EMBEDDING_DIM = 160

In [27]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words = MAX_NB_WORDS, 
                      filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~',
                      lower=True, split=' ', char_level=False, 
                      oov_token=None, document_count=0)

tokenizer.fit_on_texts(all_lines)

In [29]:
config = tokenizer.get_config()

In [30]:
config['word_counts']

{'num_words': 500,
 'filters': '!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~',
 'lower': True,
 'split': ' ',
 'char_level': False,
 'oov_token': None,
 'document_count': 10000,
 'word_counts': '{"the": 11717, "movie": 701, "begins": 94, "in": 3638, "past": 103, "where": 232, "a": 7827, "young": 298, "boy": 100, "named": 69, "sam": 35, "attempts": 40, "to": 5512, "save": 60, "celebi": 7, "from": 838, "hunter": 21, "emerging": 6, "human": 88, "psyche": 6, "and": 6811, "showing": 14, "characteristics": 3, "of": 6378, "abstract": 5, "expressionism": 1, "minimalism": 1, "russian": 24, "constructivism": 1, "graffiti": 5, "removal": 3, "has": 884, "secured": 1, "its": 784, "place": 110, "history": 69, "modern": 61, "art": 74, "while": 253, "being": 177, "created": 30, "by": 1012, "artists": 11, "who": 941, "are": 818, "unconscious": 3, "their": 774, "artistic": 21, "achievements": 4, "spurning": 1, "her": 1108, "mother\'s": 16, "insistence": 2, "that": 2107, "she": 493, "get": 247, "on": 1302, "with": 1

In [ ]:
train_sequences = tokenizer.texts_to_sequences(df_tem3['review'])
test_sequences = tokenizer.texts_to_sequences(test['review'])

## TODO
### Use emotions vectors as input to model
### Use emotions to inspect input and re-label if needed
### See if remove highest frequency words?